In [1]:
from scipy.spatial import distance
import pickle
from sklearn.metrics import classification_report
from random import shuffle
import os.path

def pickle_operating(fname, item):
    # save or load the pickle file.
    file_name = '%s.pickle' % fname
    print(file_name)
    if not item:
        with open(file_name, 'rb') as fs:
            item = pickle.load(fs)
            return item
    else:
        with open(file_name, 'wb') as fs:
            pickle.dump(item, fs, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
def getNeighbors(dataset, node, k_val=10):
    distances = []
    for x in range(len(dataset)):
        d = distance.euclidean(node, dataset[x][0])
        distances.append((x, d))
    distances.sort(key=lambda y: y[1])
    neighbors = []
    for x in range(k_val):
        neighbors.append((distances[x][0], dataset[distances[x][0]][1]))
    return neighbors

In [3]:
def saveNearestNeighbors(train_data, test_data):
    data_nn = []
    for x in range(len(test_data)):
        node = test_data[x][0]
        data_nn.append(getNeighbors(train_data, node, 10))
    return data_nn

In [4]:
def getLabels(neighbors, k_val):
    label_votes = {}
    for x in range(0, k_val):
        label = neighbors[x][1]
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    sorted_votes = sorted(label_votes.iteritems(), key=lambda y: y[1], reverse=True)
    return sorted_votes[0][0]

In [5]:
def knn_experiment(testdata, testdata_nn, k=3):
    y_classified = []
    y_true = [x[1] for x in testdata]
    for x in range(len(testdata)):
        neighbors = testdata_nn[x]
        label = getLabels(neighbors, k)
        y_classified.append(label)
        #print('%s actual= %s, predicted=%s' % (x, y_true[x], label))
    print(classification_report(y_true, y_classified))

In [6]:
dataset = pickle_operating('dataset_1', None)
del dataset['train'][0]
del dataset['test'][0]
print(len(dataset['train']), len(dataset['test']))
from random import shuffle
shuffle(dataset['train'])
shuffle(dataset['test'])

dataset_1.pickle
(320, 324)


In [7]:
data_nn = saveNearestNeighbors(dataset['train'], dataset['test'])
pickle_operating('testdata_nn', data_nn)

testdata_nn.pickle


In [8]:
testdata_nn = pickle_operating('testdata_nn', None)
knn_experiment(dataset['test'], testdata_nn, 3)

testdata_nn.pickle
             precision    recall  f1-score   support

          1       0.16      0.28      0.20        36
          2       0.25      0.26      0.25        31
          3       0.24      0.52      0.32        33
          4       0.00      0.00      0.00        21
          5       0.53      0.53      0.53        43
          6       0.71      0.22      0.33        46
          7       0.40      0.19      0.26        32
          8       0.21      0.33      0.26        27
          9       0.38      0.36      0.37        25
         10       1.00      0.53      0.70        30

avg / total       0.42      0.33      0.34       324

